In [1]:
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
import numpy as np

In [2]:
df = pd.read_csv('encoded.csv')

In [3]:
df.head()

,Age,Gender,Days Abdominal Pain,Temperature,Tenderness,Leukocytes [10^9/l],CRP [mg/l],Number of Sonographies preop,Preoperative Diagnosis
0,15,1,1,36.9,1,11.10,6.0,1,0
1,14,0,2,37.0,1,21.28,30.8,1,0
2,16,0,0,36.9,0,13.07,1.9,1,0
3,14,0,1,35.7,0,7.95,0.0,2,0
4,16,1,7,37.9,1,14.82,14.4,3,0


In [4]:
print(df.dtypes)

Age                               int64
Gender                            int64
Days Abdominal Pain               int64
Temperature                     float64
Tenderness                        int64
Leukocytes [10^9/l]             float64
CRP [mg/l]                      float64
Number of Sonographies preop      int64
Preoperative Diagnosis            int64
dtype: object


In [5]:
def mmd(X, protoypes, gamma):
    

SyntaxError: unexpected EOF while parsing (1121874355.py, line 2)

In [ ]:
def get_prototype(X, protoypes, gamma):
    cur_mmd = mmd(X, prototypes, gamma)
    decrease_mmd = float('-inf')
    
    
    

In [11]:
float('-inf')

-inf

In [13]:
def find_prototypes(X, m=10, gamma=0.1):
    chosen = pd.DataFrame()
    
    while chosen.shape[0] < m:
        chosen.append(get_prototype(X, chosen, gamma))
    
    print(chosen)

In [14]:
find_prototypes()

TypeError: find_prototypes() missing 1 required positional argument: 'X'